# 0. Librerías y funciones almacenadas

In [1]:
# Librerías
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select
import pandas as pd
import geopandas as gpd
import time
import os
from datetime import datetime
import camelot
import pickle

ModuleNotFoundError: No module named 'selenium'

In [ ]:
# Configurar preferencias
import warnings
warnings.filterwarnings('ignore')

import ctypes
from ctypes.util import find_library
find_library("".join(("gsdll", str(ctypes.sizeof(ctypes.c_voidp) * 8), ".dll")))

options = Options()
options.headless = True

options.set_preference('permissions.default.image', 2)
options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', False)
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference('pdfjs.disabled', True)
options.set_preference("browser.download.dir", "C:\\Users\\HP SUPPORT\\Documents\\trabajos\\DataMining\\pdfs2023")
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")

In [ ]:
# Funciones
def contiene_substring(dataframe, substrings,extract=False):
    for columna in dataframe.columns:
        for indice, valor in dataframe[columna].items():
            for substring in substrings:
                if substring in str(valor):
                    return substring if extract==True else True 
    return False

diccionario_meses = {
    1: "Enero",
    2: "Enero",
    3: "Enero",
    4: "Enero",
    5: "Febrero",
    6: "Febrero",
    7: "Febrero",
    8: "Marzo",
    9: "Marzo",
    10: "Marzo",
    11: "Marzo",
    12: "Marzo",
    13: "Abril",
    14: "Abril",
    15: "Abril",
    16: "Abril",
    17: "Mayo",
    18: "Mayo",
    19: "Mayo",
    20: "Mayo",
    21: "Mayo",
    22: "Junio",
    23: "Junio",
    24: "Junio",
    25: "Junio",
    26: "Julio",
    27: "Julio",
    28: "Julio",
    29: "Julio",
    30: "Julio",
    31: "Agosto",
    32: "Agosto",
    33: "Agosto",
    34: "Agosto",
    35: "Agosto",
    36: "Setiembre",
    37: "Setiembre",
    38: "Setiembre",
    39: "Setiembre",
    40: "Octubre",
    41: "Octubre",
    42: "Octubre",
    43: "Octubre",
    44: "Octubre",
    45: "Noviembre",
    46: "Noviembre",
    47: "Noviembre",
    48: "Noviembre",
    49: "Diciembre",
    50: "Diciembre",
    51: "Diciembre",
    52: "Diciembre"
}

# 1. Preparación de casos de dengue, 2020-2024

En las enumeraciones se encuentra nuestro paso a paso para recuperar la información. No obstante, no es necesario que lo cargue todo, debido a que desde 2023 es un proceso que demora horas. Para la obtención de la tabla completa, diríganse a la enumeración 1.* que está al final del punto 1.

## 1.1. Recolección de casos de dengue

### 1.1.1. Casos de incidencias de dengue en la actualidad, 2024

* FUENTE: https://www.dge.gob.pe/sala-situacional-dengue/#grafico01
* AUTOR: Centro Nacional de Epidemiología, Prevención y Control de Enfermedades (CDC) del Ministerio de Salud (MINSA) del Perú
* ÚLTIMA ACTUALIZACIÓN: 14 de abril, 2024
* ARCHIVOS DESCARGADOS: 'Sala situacional de enfermedades metaxénicas', derivado de la sección 'Según semana epidemiológica'. Departamentos: 'Lima' y 'Callao'.

Esta página creada por el CDC MINSA informa sobre las incidencias de dengue en función de la semana epidemiológica del 2024.
Por el enfoque de nuestro estudio, descargamos la información de los departamentos de Lima y El Callao, y pretendemos unificar sus registros.

In [5]:
# Importación y agrupación de los registros
lima = pd.read_excel("anexos/incidencias_dengue/lima2024.xlsx",sheet_name="Sheet1")
callao = pd.read_excel("anexos/incidencias_dengue/callao2024.xlsx",sheet_name="Sheet1")
base2024 = pd.concat([lima,callao],axis=0)

# Agregación de columna suma
base2024['Casos'] = base2024['Sin signos de alarma_total'] + base2024['Con signos de alarma_total'] + base2024['Grave_total']
base2024['Año'] = 2024
base2024 = base2024[['Año','Provincia','Distrito','Semana','Casos']] 

# Información
print(f'Dimensiones de incidencias: {base2024.shape} \nLista de atributos: {list(base2024.columns)} ') # (1131 filas, 7 columnas)
base2024.describe()

Dimensiones de incidencias: (1552, 5) 
Lista de atributos: ['Año', 'Provincia', 'Distrito', 'Semana', 'Casos'] 


,Año,Semana,Casos
count,1552.0,1552.000000,1552.000000
mean,2024.0,8.500000,32.892397
std,0.0,4.611258,88.026896
min,2024.0,1.000000,0.000000
25%,2024.0,4.750000,0.000000
50%,2024.0,8.500000,2.000000
75%,2024.0,12.250000,22.000000
max,2024.0,16.000000,906.000000


Se pueden realizar observaciones preliminares sobre el dataset:
* Contiene la totalidad de sus datos (tiene 1131 registros y cada columna cuenta de 1131 valores).
* No se deberían sumar las semanas, debido a que solo pauta de agrupación de tiempo. Los estadísticos de esa columna no dicen nada.
* Los valores de los cuartiles hasta el 75% informan que la mayor parte del dataset cuenta con valores bastante pequeño, lo que también lo confirman las desviaciones estándar grandes y positivas. La dispersión de los datos de cada variable, tal cual está el dataset, es asimétrica positiva.

### 1.1.2. Casos de incidencias de dengue 2020-2022

* FUENTE: https://www.datosabiertos.gob.pe/dataset/vigilancia-epidemiol%C3%B3gica-de-dengue
* AUTOR: Centro Nacional de Epidemiología, Prevención y Control de Enfermedades (CDC) del Ministerio de Salud (MINSA) del Perú
* ÚLTIMA ACTUALIZACIÓN: 22 de noviembre, 2023
* ARCHIVOS DESCARGADOS: 'datos_abiertos_vigilancia_dengue.csv'

In [6]:
# Cargar el dataset de 2000 al 2022
peru2000_2022 = pd.read_csv("anexos/incidencias_dengue/peru2000_2022.csv",sep=',',on_bad_lines='warn')
to2022 = peru2000_2022.loc[((peru2000_2022['departamento']=='LIMA') | (peru2000_2022['departamento']=='CALLAO')) & ((peru2000_2022['ano'] <= 2022) & (peru2000_2022['ano'] >= 2020))]
to2022 = to2022[['ano','provincia','distrito','semana']]
to2022['casos'] = 1
to2022.set_index(['ano','provincia','distrito','semana'],inplace=True)

# Agruparlo al formato de trabajo
base2022 = to2022.groupby(level=[0,1,2,3])['casos'].sum().reset_index()
base2022.columns = ['Año','Provincia','Distrito','Semana','Casos']
base2022

Skipping line 87871: expected 14 fields, saw 16
Skipping line 88799: expected 14 fields, saw 16
Skipping line 89573: expected 14 fields, saw 16

Skipping line 176478: expected 14 fields, saw 16
Skipping line 177184: expected 14 fields, saw 16
Skipping line 177191: expected 14 fields, saw 16
Skipping line 183099: expected 14 fields, saw 16

Skipping line 293518: expected 14 fields, saw 16



,Año,Provincia,Distrito,Semana,Casos
0,2020,CALLAO,VENTANILLA,8,2
1,2020,CALLAO,VENTANILLA,10,1
2,2020,CALLAO,VENTANILLA,14,1
3,2020,LIMA,ATE,10,1
4,2020,LIMA,ATE,15,1
...,...,...,...,...,...
473,2022,LIMA,VILLA MARIA DEL TRIUNFO,18,4
474,2022,LIMA,VILLA MARIA DEL TRIUNFO,19,4
475,2022,LIMA,VILLA MARIA DEL TRIUNFO,20,2
476,2022,LIMA,VILLA MARIA DEL TRIUNFO,21,1


### 1.1.3. Casos de incidencias de dengue 2023

* FUENTE: https://www.dge.gob.pe/portalnuevo/informacion-publica/situacion-del-dengue-en-el-peru/
* AUTOR: Centro Nacional de Epidemiología, Prevención y Control de Enfermedades (CDC) del Ministerio de Salud (MINSA) del Perú
* ÚLTIMA ACTUALIZACIÓN: 29 de diciembre, 2023
* ARCHIVOS DESCARGADOS: Todos los .pdf del año 2023

In [ ]:
# Crear instancia del navegador
driver = webdriver.Firefox(options=options)
driver.implicitly_wait(300)

# Abrir url en el driver
driver.get('https://www.dge.gob.pe/epipublic/publico/listado/13/2023/0')
driver.maximize_window()
time.sleep(5)

# Colocar 50 registros por página
selectPaginacion = Select(driver.find_element(By.XPATH, '//select[@name="casosIndividual_length"]'))
selectPaginacion.select_by_visible_text('50')
time.sleep(2)

# Tablas
filas = driver.find_elements(By.XPATH, '//a[text()="Ver archivo"]')
for fila in filas:
    fila.click()

In [ ]:
# Cambiar nombre de los pdfs
for pdf in os.listdir('pdfs2023'):
    if len(pdf)<=17: pass
    else: os.rename(f"anexos/pdfs2023/{pdf}",f"pdfs2023/{pdf.split('_')[0] + '_' + pdf.split('_')[1]}.pdf")

# EXCLUSIVAMENTE QUE CONTENGAN LAS PALABRAS CLAVES 'LIMA', 'Lima', 'Callao' y '2022 - 2023*'
pdfscraping = {'semana':[],
               'tablas':[]}
claves = ['LIMA','Lima','Callao','2022* - 2023*']

for i in range(1,53):
    if os.path.exists(f'anexos/pdfs2023/dengue_2023{i}.pdf'):
        tables = camelot.read_pdf(f'anexos/pdfs2023/dengue_2023{i}.pdf',flavor='stream',pages='all')
        tablas_interes = [tabla.df for tabla in tables if any(tabla.df.apply(lambda column: column.astype(str).str.contains('|'.join(claves)).any()))]
        
        pdfscraping['semana'] += [i]
        pdfscraping['tablas'] += [tablas_interes]
    else: pass

    if i in [5,10,15,20,25,30,35,40,45,50]:
        print(f'Checkpoint {i}: {datetime.now()}')

# Guardarlo para reusarlo en otra sesión
with open("anexos/pdfs2023/pdfscraping.obj",'wb') as filehandler:
    pickle.dump(pdfscraping,filehandler)

# Revisar cuantas tablas se tienen
acumulados = {'n_tablas':[]}
for i in range(len(pdfscraping['semana'])):
    acumulados['n_tablas'] += [len(pdfscraping['tablas'][i])]

control = pd.DataFrame(acumulados,index=pdfscraping['semana'])
control.sum()

In [ ]:
# Seteo de las variables y los diccionarios
agrups = ['Lima Region','Callao','Lima Metropolitana','DIRIS Lima Norte','DIRIS Lima Este','DIRIS Lima Sur','DIRIS Lima Centro']
print('Inicio procesamiento: ', datetime.now())

hemeroteca = []
# Bucle de limpieza de tablas por cada año
for i in range(len(pdfscraping['tablas'])):
    tablas = pdfscraping['tablas'][i]
    sem = f"SE {str(pdfscraping['semana'][i]).zfill(2)}"
    for tabla in tablas:
        # Buscar inicio y final de la tabla
        indexacion = 0
        for col in range(tabla.shape[1]):
            for indice, valor in tabla.iloc[:,col].items():
                if valor.isupper():
                    indexacion = indice
                    break

        # Índice de columna desde SE
        poscol = 0
        for row in range(tabla.shape[0]):
            for column in range(len(tabla.columns)):
                if sem in tabla.iloc[row,column]:
                    poscol = column
                    break

        # Agrupar tabla en las direcciones distritales        
        agrupar = contiene_substring(tabla,agrups,True)
        
        # Transformar tabla        
        transtable = pd.concat([tabla.iloc[:,0],tabla.iloc[:, poscol]],axis=1,ignore_index=True)
        transtable = transtable.iloc[indexacion:,:]
        transtable.columns = ['Distrito','Casos']
        
        # Agregar etiquetas
        transtable['Semana'] = pdfscraping['semana'][i]
        transtable['Región'] = agrupar

        hemeroteca += [transtable]
    
    if i in [5,10,15,20,25,30,35,40,45,50]:
        print(f'Checkpoint {i}: {datetime.now()}')

# Guardar variable como backup
with open("anexos/pdfs2023/tablas.obj",'wb') as filehandler: # Guardarlo para reusarlo en otra sesión
    pickle.dump(hemeroteca,filehandler)

In [7]:
# No se rehace todo el largo proceso: abre el archivo pickle
with open("anexos/pdfs2023/tablas.obj",'rb') as filehandler:
    hemeroteca = pickle.load(filehandler)

dfs = []
for lista_df in dict(hemeroteca).values():
    dfs.extend(lista_df)

In [8]:
# limpieza de la tabla recopilada de los pdfs
semanas = pd.concat(dfs, axis=0,ignore_index=True)
semanas = semanas.loc[~(semanas['Distrito']=='') & (~semanas['Distrito'].str.contains('Total'))]
semanas['Casos'] = pd.to_numeric(semanas['Casos'], errors='coerce')
semanas = semanas[pd.notnull(semanas['Casos'])]

distrital = gpd.read_file('anexos/distritos/distritos-peru@bogota-laburbano.geojson')
distrital = distrital[(distrital['nombdep']=='LIMA')|(distrital['nombdep']=='CALLAO')][['nombdist','nombprov']]
distrital = distrital.iloc[:-1,:]

distrital.columns = ['Distrito','Provincia']
semanas = semanas.merge(distrital,how='left',on="Distrito")
semanas

,Distrito,Casos,Semana,Región,Provincia
0,SAN JUAN DE LURIGANCHO,1.0,1,False,LIMA
1,SANTIAGO DE SURCO,1.0,1,False,LIMA
2,ATE,2.0,1,False,LIMA
3,LURIGANCHO,2.0,1,False,LIMA
4,COMAS,0.0,1,False,LIMA
...,...,...,...,...,...
2503,CHILCA,6.0,52,Lima Region,CAÑETE
2504,IMPERIAL,0.0,52,Lima Region,CAÑETE
2505,SAN VICENTE DE CAÑETE,0.0,52,Lima Region,CAÑETE
2506,ASIA,1.0,52,Lima Region,CAÑETE


#### Observaciones con la tabla total

Tablas vacías no correspondientes:
* BARRANCA solo ha tenido 3 casos: 1 en PARAMONGA, 2 en SUPE.
* CAJATAMBO solo ha tenido 2 casos registrados en GORGOR.
* CANTA ha tenido solo dos casos en HUAMANTANGA
* HUARAL solo ha tenido 4 casos: 3 en HUARAL, 1 en AUCALLAMA
* HUAURA ha tenido 3 casos: 2 en Huacho y 1 en Sayan.

Tablas vacías correctas:
* OYON no tiene registros
* YAUYOS no tiene registritos

Tablas que arreglar:
* HUAROCHIRI tiene un total de casos 4 valores menor que el consolidado de 2023.
* CALLAO tiene muchos registros con cantidad de casos 0.
* LIMA tiene muchos registros duplicados.

In [9]:
provincias = list(distrital['Provincia'].value_counts().index)

for prov in provincias:
    ejecutar = f"""
{prov} = semanas[semanas['Distrito'].isin(distrital[distrital['Provincia']=='{prov}']['Distrito'])]
"""
    exec(ejecutar)

LIMA = LIMA[(LIMA['Provincia']=='LIMA')&(LIMA['Casos']!=0)]
LIMA.drop_duplicates(subset=['Distrito','Semana'],inplace=True)

HUAROCHIRI = HUAROCHIRI[(HUAROCHIRI['Provincia']=='HUAROCHIRI')&(HUAROCHIRI['Casos']!=0)]
HUAROCHIRI.at[895,'Casos'] = 28 # (+4) En otras fuente, 
# sale que San Antonio en Huarochiri concentra una gran cantidad de casos. Por ello, para cubrir sus casos totales,
# se añadirán a su semana con más casos

CALLAO = semanas[semanas['Distrito'].isin(distrital[distrital['Provincia']=='CALLAO']['Distrito'])]

agreg = {
    'Provincia': ['BARRANCA', 'BARRANCA', 'CAJATAMBO', 'CANTA', 'HUARAL', 'HUARAL', 'HUAURA', 'HUAURA'],
    'Distrito': ['PARAMONGA', 'SUPE', 'GORGOR', 'HUAMANTANGA', 'HUARAL', 'AUCALLAMA', 'HUACHO', 'SAYAN'],
    'Semana': [20, 20, 10, 18, 17, 42, 21, 22],
    'Casos': [1, 2, 2, 2, 3, 1, 2, 1]
}

tabla2023 = pd.concat([LIMA, CALLAO, HUAROCHIRI,pd.DataFrame(agreg)],axis=0,ignore_index=True)
tabla2023['Año'] = 2023
base2023 = tabla2023.sort_values(['Semana']).reset_index()[['Año','Provincia','Distrito','Semana','Casos']]
base2023.to_csv('anexos/incidencias_dengue/2023.csv')

print('Dimensiones de la tabla: ',base2023.shape)
print('Cantidad de casos acumulados en 2023: ',base2023['Casos'].sum())

Dimensiones de la tabla:  (1025, 5)
Cantidad de casos acumulados en 2023:  16464.0


### 1.1.4. Concatenación y preparación de las tablas

In [10]:
# Concatenar tablas
base = pd.concat([base2022,base2023,base2024],axis=0,ignore_index=True)
base['Semana'].replace(diccionario_meses,inplace=True)
base.drop(base[base['Semana']==53].index,inplace=True)
base.columns = ['Año','Provincia','Distrito','Mes','Casos']

# Agrupar casos por meses
base.set_index(['Año','Provincia','Distrito','Mes'],inplace=True)
base = base.groupby(level=[0,1,2,3])['Casos'].sum()
base = base.reset_index()

In [11]:
# Agregar filas con casos igual a 0
pairs = {(row['Provincia'], row['Distrito']) for _, row in distrital.iterrows()}

años = [2020,2021,2022,2023,2024]
meses = list(diccionario_meses.values())

for año in años:
    print(año, ' - ', datetime.now())
    for mes in meses:
        for provincia, distrito in pairs:
            registro = {'Año':año,'Provincia':provincia,'Distrito':distrito,'Mes':mes,'Casos':0}
            base.loc[base.shape[0]] = registro

base.drop_duplicates(subset=['Año','Provincia','Distrito','Mes'],inplace=True,ignore_index=True)
base.drop(base[(base['Año']==2024) & (base['Mes'].isin(['Mayo','Junio','Julio','Agosto','Setiembre','Octubre','Noviembre','Diciembre']))].index,inplace=True)
base

2020  -  2024-04-28 22:17:11.933329
2021  -  2024-04-28 22:17:34.647983



KeyboardInterrupt



Ya se tienen ahora todos los distritos mapeados para la serie temporal. En la exploración, se observa que una 'ñ' de Breña no se registró bien, de modo que se corregirá.

In [30]:
base.iloc[3136,4] = 1
base = base.drop(66)

base['Distrito'].value_counts()

Distrito
SAN LUIS       104
SAN ANTONIO    104
LARAOS         104
MIRAFLORES     104
VIÑAC           52
              ... 
SAN BARTOLO     52
SANTA ROSA      52
SURQUILLO       52
BARRANCA        52
NAVAN           52
Name: count, Length: 174, dtype: int64

In [31]:
# Exportación
base.to_csv('datasets/dengue.csv',index=False)

### 1.1.* Importación directa de tabla completa

In [28]:
base = pd.read_csv('datasets/dengue.csv')
base

,Año,Provincia,Distrito,Mes,Casos
0,2020,CALLAO,VENTANILLA,Abril,1.0
1,2020,CALLAO,VENTANILLA,Marzo,3.0
2,2020,LIMA,ATE,Abril,3.0
3,2020,LIMA,ATE,Marzo,1.0
4,2020,LIMA,ATE,Mayo,21.0
...,...,...,...,...,...
9251,2024,YAUYOS,COCHAS,Abril,0.0
9252,2024,YAUYOS,ALLAUCA,Abril,0.0
9253,2024,HUARAL,PACARAOS,Abril,0.0
9254,2024,HUAROCHIRI,SAN LORENZO DE QUINTI,Abril,0.0


## 1.2. Importación y preparación de archivos ShapeFile para la visualización provincial y distrital del Perú.

* FUENTE: https://www.geogpsperu.com/2018/02/limite-provincial-politico-shapefile.html
* AUTOR: GEO GPS PERÚ (recopilado del Instituto Nacional de Estadística)
* ÚLTIMA ACTUALIZACIÓN: 2023
* ARCHIVOS DESCARGADOS: 'Límites_distritales.rar' y 'Límites_provinciales.rar' (INEI, 2023)

Los archivos descargados contienen la información poligonal de los distritos y las provincias de todo el Perú, que luego será utilizada para realizar el análisis espacial pertinente en el API de OpenStreetMaps. La variable de interés para la tarea es 'geometry'; las de unión, 'provincia' y 'distrito'.

In [102]:
# Importación de valores geométricos para visualización espacial

distrital = gpd.read_file('anexos/distritos/distritos.shp')
print(f'Dimensiones de distritos.shp: {distrital.shape} \nLista de atributos: {list(distrital.columns)} ') # (1891 filas, 10 columnas)

Dimensiones de distritos.shp: (1891, 10) 
Lista de atributos: ['UBIGEO', 'CCDD', 'CCPP', 'CCDI', 'DEPARTAMEN', 'PROVINCIA', 'DISTRITO', 'OBJECTID', 'ESRI_OID', 'geometry'] 


### Registros borrados

En ambas tablas, nos quedaremos exclusivamente con los registros que corresponden a los departamentos de Lima y El Callao. Para ello, se realiza una selección de filas donde los valores de las columnas 'DEPARTAMEN' sean 'LIMA' o 'CALLAO'.

### Columnas mantenidas
* DISTRITO (solo tabla distrito): etiqueta de nombre de distrito.
* PROVINCIA: etiqueta de nombre de provincia.
* geometry: contiene la información geométrica de la zona.

Asimismo, se pasarán los nombres de las columnas a minúsculas para mantener integridad de nombre de columnas con las tablas que se trabajarán más adelante.

In [105]:
# Pasando los nombres de las columnas a minúsculas

distrital.columns = distrital.columns.str.lower()
#base.columns = base.columns.str.lower()

# Selección de registros y proyección de atributos de interés

distrital = distrital.loc[((distrital['departamen']=='LIMA') | (distrital['departamen']=='CALLAO'))].reset_index()
distrital = distrital[['objectid','provincia','distrito','geometry']]

print(f'Dimensiones de distrital: {distrital.shape}') # (178 filas, 4 columnas)

Dimensiones de distrital: (178, 4)


In [18]:
# Completitud de datos en distritos
print(f'Completitud de datos en distritos (filas totales: 178): \n{distrital.count()} \n\nCompletitud de datos en provincial (filas totales: 11): \n{provincial.count()}')

Completitud de datos en distritos (filas totales: 178): 
objectid     178
provincia    178
distrito     178
geometry     178
dtype: int64 

Completitud de datos en provincial (filas totales: 11): 
objectid     11
provincia    11
geometry     11
dtype: int64


Ningún dato faltante en ambas tablas. Podemos ya trabajar con estas.

La totalidad de los datos de las dos tablas se agrupan satisfactoriamente.

# 2. Variables climatológicas

In [1]:
import os
import pandas as pd
import re
from io import StringIO

def read_concat(prefijo: str):
    dfs = []
    
    for filename in os.listdir('anexos/clima'):
        if filename.startswith(prefijo) and filename.endswith('.csv'):
            dfs.append(pd.read_csv(os.path.join('anexos/clima',filename)))
            
    concat_df = pd.concat(dfs,ignore_index=True)
    return concat_df

def segment(df):
    return

def fix_date(df):
    month_map = {
        'enero': '01',
        'febrero': '02',
        'marzo': '03',
        'abril': '04',
        'mayo': '05',
        'junio': '06',
        'julio': '07',
        'agosto': '08',
        'septiembre': '09',
        'octubre': '10',
        'noviembre': '11',
        'diciembre': '12'
    }
    df['Fecha'] = df.apply(lambda row: f"{row['Año']}-{month_map[row['Mes']]}-{str(row['Día']).zfill(2)}", axis=1)
    columns = ['Fecha'] + [col for col in df.columns if col not in ['Año', 'Mes', 'Día', 'Fecha']]
    df = df[columns]
    
    return df

def extract_number(value):
    if pd.isna(value):
        return value
    value = value.replace(',', '.').replace('−', '-')

    return float(value.split()[0])

def transform_numbers(df):
    df['Min'] = df['Min'].apply(extract_number)
    df['Max'] = df['Max'].apply(extract_number)
    return df

def clean_csv(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    cleaned_lines = []
    buffer = ""
    for index, line in enumerate(lines):
        if index == 0:
            cleaned_lines.append(line.strip())  # Agregar la primera línea como encabezado
            continue
        if re.match(r'^\d{4}-\d{2}-\d{2}', line):
            if buffer:
                cleaned_lines.append(buffer)
            buffer = line.strip()
        else:
            buffer += " " + line.strip()

    # Añadir la última línea procesada
    if buffer:
        cleaned_lines.append(buffer)

    # Crear un dataframe a partir de las líneas limpias
    cleaned_data = "\n".join(cleaned_lines)
    df = pd.read_csv(StringIO(cleaned_data))
    return df

def imputar_datos(df):
    # Convertir la columna de fecha a datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    
    # Extraer el año y mes
    df['Año'] = df['Fecha'].dt.year
    df['Mes'] = df['Fecha'].dt.month
    
    # Seleccionar solo las columnas numéricas
    numeric_cols = df.select_dtypes(include=['number']).columns
    
    # Agrupar por año, mes, provincia y distrito
    grouped = df.groupby(['Año', 'Mes', 'Provincia', 'Distrito'])
    
    # Imputar valores nulos con la media de cada grupo solo en columnas numéricas
    df_imputed = df.copy()
    df_imputed[numeric_cols] = grouped[numeric_cols].transform(lambda x: x.fillna(x.mean()))
    
    # Eliminar las columnas 'Año' y 'Mes' añadidas para la agrupación
    df_imputed.drop(columns=['Año', 'Mes'], inplace=True)
    
    return df_imputed

def merge_dfs(dfs, on_columns):
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = merged_df.merge(df, on=on_columns, how='outer')
    return merged_df

## 2.1. Limpieza de cada datasets de variables climatológicas scrapeadas

In [2]:
import numpy as np

# ------------
# Rain-3h

rain_3h = clean_csv(r'anexos\clima\rain-3h.csv')
rain_3h['Min_text'] = rain_3h.apply(lambda row: row['Min'].split(' ')[-1] if isinstance(row['Min'], str) and row['Min'].split(' ')[-1] != 'mm' else 'Despejado', axis=1)
rain_3h['Max_text'] = rain_3h.apply(lambda row: row['Max'].split(' ')[-1] if isinstance(row['Max'], str) and row['Max'].split(' ')[-1] != 'mm' else 'Despejado', axis=1)
rain_3h['Weather'] = np.where(
    (rain_3h['Min_text'] == 'Despejado') & (rain_3h['Max_text'] == 'Despejado'), 'Despejado',
    np.where(rain_3h['Min_text'] != 'Despejado', rain_3h['Min_text'], rain_3h['Max_text'])
)
rain_3h = transform_numbers(rain_3h)
rain_3h = imputar_datos(rain_3h)
rain_3h = rain_3h.filter(['Fecha','Semana','Provincia','Distrito','Weather','Min','Max'])
rain_3h.columns = ['Fecha','Semana','Provincia','Distrito','Clima','Pluvio_Min','Pluvio_Max']

# ------------
# Temperature-2m

temp = read_concat('temperature-2m')
temp = imputar_datos(transform_numbers(temp))
temp.columns = ['Fecha','Semana','Provincia','Distrito','Temp_Min','Temp_Max']

# ------------
# Clouds-total

clouds = read_concat('clouds-total')
clouds = imputar_datos(transform_numbers(clouds))
clouds['Min'] = clouds['Min']/100
clouds = clouds.filter(['Fecha','Semana','Provincia','Distrito','Min'])
clouds.columns = ['Fecha','Semana','Provincia','Distrito','Nubosidad']

# ------------
# Wind-10m

wind = fix_date(read_concat('wind-10m'))
wind = imputar_datos(transform_numbers(wind))
wind.columns = ['Fecha','Provincia','Distrito','Semana','Viento_Min','Viento_Max']

# ------------
# Pressure

pressure = fix_date(read_concat('pressure'))
pressure = imputar_datos(transform_numbers(pressure))
pressure.drop(columns=['Max'],inplace=True)
pressure.columns = ['Fecha','Provincia','Distrito','Semana','Presión']

# ------------
# Dew

dew = pd.read_csv('anexos/clima/dew.csv')
dew = imputar_datos(transform_numbers(dew))
dew.drop(columns=['Max'],inplace=True)
dew.columns = ['Fecha','Semana','Provincia','Distrito','Punto_Rocío']

## 2.2. Concatenación de los datos en un solo dataset metereológico

Comprendiendo solo Lima Metropolitana y El Callao.

In [3]:
dfs = [rain_3h,clouds,pressure,wind,temp,dew]
columns = ['Fecha','Semana','Provincia','Distrito']

clima = merge_dfs(dfs,columns)
clima = clima.loc[clima['Provincia'].isin(['LIMA','CALLAO']),:]
clima.describe()

,Fecha,Semana,Pluvio_Min,Pluvio_Max,Nubosidad,Presión,Viento_Min,Viento_Max,Temp_Min,Temp_Max,Punto_Rocío
count,16450,16450.000000,13000.000000,13000.000000,10920.000000,12990.000000,13000.000000,13000.000000,12450.000000,13000.000000,13000.000000
mean,2022-02-28 03:30:05.471124736,25.112462,0.014367,0.000394,0.564348,1010.888395,5.858724,12.495038,17.719351,21.750154,15.971929
min,2020-01-01 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-128.000000,14.000000,-1.000000
25%,2021-02-07 00:00:00,11.000000,0.000000,0.000000,0.300000,1012.000000,3.000000,11.000000,15.000000,19.000000,14.000000
50%,2022-02-22 00:00:00,24.000000,0.000000,0.000000,0.600000,1014.000000,5.000000,12.000000,18.000000,22.000000,16.000000
75%,2023-03-16 00:00:00,39.000000,0.000000,0.000000,0.800000,1015.000000,8.000000,14.000000,20.000000,24.000000,18.000000
max,2024-04-30 00:00:00,53.000000,4.300000,0.800000,1.000000,1020.000000,254.000000,25.000000,24.000000,31.000000,24.000000
std,NaN,15.503666,0.110468,0.009956,0.288507,48.683563,3.833698,2.724352,3.087386,3.392340,2.968127


In [4]:
clima.to_csv(r'datasets\variables-climatológicas\metereologia.csv',index=False)